In [21]:
import tensorflow as tf
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem

from sklearn.model_selection import train_test_split

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten, \
    Activation, BatchNormalization, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [23]:
df = pd.read_csv('cmpd.csv')
count = 0

train = pd.DataFrame(columns=['inchikey', 'smiles', 'activity'])
test = pd.DataFrame(columns=['inchikey', 'smiles', 'activity'])

for idx in range(len(df)):
    if df.iloc[idx, -2] == 'train':
        train.loc[idx] = df.iloc[idx, [0, 1, 3]]
        count += 1
    else:
        test.loc[idx] = df.iloc[idx, [0, 1, 3]]
        count += 1

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [24]:
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1].to_list()
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1].to_list()

In [25]:
def y_to_vec(list):
    for idx in range(len(list)):
        if list[idx] == 'active':
            list[idx] = 1
        else:
            list[idx] = 0
    return list

y_train = y_to_vec(y_train)
y_test = y_to_vec(y_test)

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.9, random_state=23)

print(len(X_train)) # 3181
print(len(X_val))   # 796
print(len(X_test))  # 1553

print(len(y_train)) # 3181
print(len(y_val))   # 796
print(len(y_test))  # 1553

3579
398
1553
3579
398
1553


In [27]:
def make_dataset(df):
    temp_list = []

    for idx in range(len(df)):
        temp_list.append(df.iloc[idx, :])
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
    tokenizer.fit_on_texts(temp_list)
    top_k = len(tokenizer.word_index)

    temp_seq = tokenizer.texts_to_sequences(temp_list)
    temp_vec = tf.keras.preprocessing.sequence.pad_sequences(temp_seq, padding='post')
    return temp_vec

X_train = make_dataset(X_train)
X_val = make_dataset(X_val)
X_test = make_dataset(X_test)

In [28]:
print(X_train.shape)
print(len(np.unique(X_train)))

(3579, 2)
7121


In [29]:
es = EarlyStopping(mode='auto', patience=30, monitor='val_loss', restore_best_weights=True)
rl = ReduceLROnPlateau(mode='auto',monitor='val_loss',patience=10)

In [34]:
model = Sequential()
model.add(Embedding(input_dim=len(np.unique(X_train)), output_dim=11, input_length=2))
# model.add(LSTM(32))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(8))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(
    optimizer=Adam(lr=0.1),
    loss='binary_crossentropy',
    metrics='acc'
)

model.fit(
    X_train, y_train,
    validation_data=[X_val, y_val],
    epochs=100, batch_size=64,
    callbacks=[es, rl]
)

D:\APPL\anaconda\envs\bionsight\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
56/56 [==============================] - 1s 10ms/step - loss: 0.7010 - mse: 0.2526 - val_loss: 0.7249 - val_mse: 0.2618 - lr: 0.1000
Epoch 2/100
56/56 [==============================] - 0s 8ms/step - loss: 0.6912 - mse: 0.2490 - val_loss: 0.6913 - val_mse: 0.2491 - lr: 0.1000
Epoch 3/100
56/56 [==============================] - 0s 8ms/step - loss: 0.6916 - mse: 0.2492 - val_loss: 0.6927 - val_mse: 0.2498 - lr: 0.1000
Epoch 4/100
56/56 [==============================] - 0s 8ms/step - loss: 0.6919 - mse: 0.2494 - val_loss: 0.6920 - val_mse: 0.2494 - lr: 0.1000
Epoch 5/100
56/56 [==============================] - 0s 8ms/step - loss: 0.6907 - mse: 0.2488 - val_loss: 0.6920 - val_mse: 0.2494 - lr: 0.1000
Epoch 6/100
56/56 [==============================] - 0s 8ms/step - loss: 0.6927 - mse: 0.2497 - val_loss: 0.6959 - val_mse: 0.2514 - lr: 0.1000
Epoch 7/100
56/56 [==============================] - 0s 8ms/step - loss: 0.6925 - mse: 0.2496 - val_loss: 0.6982 - val_mse: 0.2525 - lr

In [35]:

loss, acc = model.evaluate(
    X_test, y_test
)


y_pred = model.predict(y_test)[:, 1]
for idx in range(len(y_pred)):
    if y_pred[idx] > 0.5:
        y_pred[idx] = 1.
    else:
        y_pred[idx] = 0.
_y_test = np.max(y_test, axis=1)

print(y_pred)
print(_y_test)

acc_score = accuracy_score(_y_test, y_pred)
recall = recall_score(_y_test, y_pred)
precision = precision_score(_y_test, y_pred)
f1 = f1_score(_y_test, y_pred)

print(loss, acc)
print(acc_score)
print(recall)
print(precision)
print(f1)
# log_loss = metrics.log_loss(np.max(y_test, axis=1), y_pred, labels=[0, 1])
# print(log_loss)

49/49 [==============================] - 0s 3ms/step - loss: 0.7114 - mse: 0.2591
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
0.7113742828369141 0.25909969210624695
1.0
1.0
1.0
1.0


In [32]:
# precision, recall, _ = metrics.precision_recall_curve(np.max(y_test, axis=1), y_pred, pos_label=None)
# fpr_roc, tpr_roc, _ = metrics.roc_curve(np.max(y_test, axis=1), y_pred, pos_label=None)

# auc1 = metrics.auc(recall, precision)
# auc2 = metrics.auc(fpr_roc, tpr_roc)

# print(auc1)
# print(auc2)

In [33]:
# train val 0.9 (BatchNormal)
# 5.406352996826172 0.5251256227493286

# train val 0.9 (Dropout)
# 31.515090942382812 0.5175879597663879

# train val 0.9 (BatchNormal + Dropout)
# 8.611763000488281 0.5150753855705261

# train val 0.9 (BatchNormal + adjust node)
# 3.570016622543335 0.5025125741958618